In [1]:
!pip install Keras-Preprocessing
from keras_preprocessing.image import ImageDataGenerator
from keras.layers import Dense, AveragePooling2D, Flatten
from keras.models import Model, Sequential
from keras.optimizers import Adam
from sklearn.metrics import f1_score, accuracy_score, confusion_matrix, classification_report, ConfusionMatrixDisplay
import matplotlib.pyplot as plt
import tensorflow as tf
import numpy as np
from google.colab import drive
drive.mount('/content/drive')


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.6/42.6 kB 523.9 kB/s eta 0:00:00
Mounted at /content/drive


In [2]:
# !pip install split-folders

In [3]:
# import splitfolders
# splitfolders.fixed(
#     "/content/drive/MyDrive/skripsi/train",
#     output="/content/drive/MyDrive/old_ratio",
#     seed=1337,
#     fixed=(200, 50),
#     oversample=False,
#     group_prefix=None
# )


In [4]:
train_dir = '/content/drive/MyDrive/old_ratio/train'
val_dir = '/content/drive/MyDrive/old_ratio/val'
test_dir = '/content/drive/MyDrive/old_ratio/test'


In [5]:
def f1_metric(y_true, y_pred):
  def recall(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0,1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0,1)))
    recall = (true_positives + K.epsilon())/ (possible_positives + K.epsilon())
    return recall
  def precision(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0,1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = (true_positives + K.epsilon()) / ( predicted_positives + K.epsilon())
    return precision
  precision_value = precision(y_true, y_pred)
  recall_value = recall(y_true, y_pred)
  f1_score = 2 * ((precision_value * recall_value) / (precision_value + recall_value + K.epsilon()))
  return f1_score

In [6]:
nilai_bs = 4
nilai_lr = 0.0001
IMG_SIZE = 224
train_datagen = ImageDataGenerator(
    rescale=1./255.,
    rotation_range=10,
    width_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
)
train = train_datagen.flow_from_directory(
    train_dir,
    class_mode="categorical",
    target_size=(IMG_SIZE, IMG_SIZE),
    batch_size=nilai_bs,
)
validation = train_datagen.flow_from_directory(
    val_dir,
    shuffle=False,
    class_mode="categorical",
    target_size=(IMG_SIZE, IMG_SIZE),
    batch_size=nilai_bs,
)


Found 762 images belonging to 2 classes.
Found 400 images belonging to 2 classes.


In [7]:
base_model = tf.keras.applications.ResNet50(
    include_top=False,
    weights="imagenet",
    input_shape=(IMG_SIZE, IMG_SIZE, 3),
)
model = Sequential([
    base_model,
    AveragePooling2D(pool_size=(1, 1)),
    Flatten(),
    Dense(2, activation="softmax")
])


94765736/94765736 [==============================] - 0s 0us/step


In [8]:
import tensorflow as tf
from tensorflow.keras import backend as K

K.clear_session()


In [ ]:
Optimizer = tf.keras.optimizers.Adam(learning_rate=nilai_lr)
model.compile(
    loss="binary_crossentropy",
    optimizer=Optimizer,
    metrics=['accuracy', f1_metric]
)
nilai_epoch = 30
nilai_patience = 8
my_callback = tf.keras.callbacks.EarlyStopping(
    monitor='val_f1_metric',
    patience=nilai_patience
)
history = model.fit(
    train,
    epochs=nilai_epoch,
    batch_size=nilai_bs,
    validation_data=validation,
    callbacks=my_callback,
)


Epoch 1/30
191/191 [==============================] - 704s 3s/step - loss: 0.7182 - accuracy: 0.7743 - f1_metric: 0.7749 - val_loss: 1.0631 - val_accuracy: 0.4800 - val_f1_metric: 0.4800
Epoch 2/30
191/191 [==============================] - 690s 4s/step - loss: 0.4988 - accuracy: 0.8793 - f1_metric: 0.8796 - val_loss: 3.1278 - val_accuracy: 0.5000 - val_f1_metric: 0.5000
Epoch 3/30
191/191 [==============================] - 634s 3s/step - loss: 0.4385 - accuracy: 0.8885 - f1_metric: 0.8887 - val_loss: 1.6920 - val_accuracy: 0.5400 - val_f1_metric: 0.5400
Epoch 4/30
191/191 [==============================] - 628s 3s/step - loss: 0.3743 - accuracy: 0.8911 - f1_metric: 0.8914 - val_loss: 0.4650 - val_accuracy: 0.8000 - val_f1_metric: 0.8000
Epoch 5/30
117/191 [=================>............] - ETA: 3:29 - loss: 0.2566 - accuracy: 0.9145 - f1_metric: 0.9145

In [ ]:
best_loss = min(history.history['loss'])
best_accuracy = max(history.history['accuracy'])
best_val_loss = min(history.history['val_loss'])
best_val_accuracy = max(history.history['val_accuracy'])

In [ ]:
import pandas as pd

In [ ]:
print("Best Training Accuracy: %.4f" % best_accuracy)
print("Best Loss: %.4f" % best_loss)
print("Best Validation Accuracy: %.4f" % best_val_accuracy)
print("Best Validation Loss: %.4f" % best_val_loss)

df_hist = pd.DataFrame(history.history)
hist_csv_file = '/content/drive/MyDrive/skripsi/Skenario_30.csv'
with open(hist_csv_file, mode='w') as f:
    df_hist.to_csv(f)
model.save('/content/drive/MyDrive/skripsi/Skenario_30.h5')


In [ ]:
def plot_metric(history, metrics):
  for metric in metrics:
    train_metrics = history.history[metric]
    val_metrics = history.history['val_'+metric]
    epochs = range(1, len(train_metrics) + 1)
    plt.plot(epochs, train_metrics)
    plt.plot(epochs, val_metrics)
    plt.title("Training and Validation " + metric)
    plt.xlabel("Epochs")
    plt.ylabel(metric)
    plt.legend(["train_"+metric, "val_"+metric])
    plt.show()
plot_metric(history, ["accuracy", "loss"])


In [ ]:
test_datagen = ImageDataGenerator(rescale=1./255.)
test_generator = test_datagen.flow_from_directory(
    test_dir,
    shuffle=False,
    target_size=(IMG_SIZE, IMG_SIZE),
    batch_size=nilai_bs,
    class_mode='categorical'
)

Y_pred = model.predict(test_generator, workers=0)
y_pred = np.argmax(Y_pred, axis=1)
y_true = test_generator.classes[test_generator.index_array]

print("Confusion Matrix")
cm = confusion_matrix(y_true, y_pred)
cm_display = ConfusionMatrixDisplay(
    confusion_matrix=cm,
    display_labels=test_generator.class_indices
)
cm_display.plot(cmap="OrRd")
plt.show()

target_names = test_generator.class_indices
print("Hasil Klasifikasi")
print(classification_report(y_true, y_pred, target_names=target_names))


In [ ]:
import cv2
from keras.models import load_model
from keras.preprocessing import image
import numpy as np

In [ ]:
import cv2
from keras.models import load_model
from keras.preprocessing import image
import numpy as np

# Fungsi f1_metric
def f1_metric(y_true, y_pred):
    true_positives = np.sum(np.round(np.clip(y_true * y_pred, 0, 1)))
    possible_positives = np.sum(np.round(np.clip(y_true, 0, 1)))
    predicted_positives = np.sum(np.round(np.clip(y_pred, 0, 1)))

    precision = true_positives / (predicted_positives + 1e-7)
    recall = true_positives / (possible_positives + 1e-7)

    f1_score = 2 * (precision * recall) / (precision + recall + 1e-7)
    return f1_score

# Path ke model yang telah dilatih
model_path = '/content/drive/MyDrive/skripsi/Skenario_30.h5'

# Memuat model dengan menyertakan fungsi khusus (f1_metric)
model = load_model(model_path, custom_objects={'f1_metric': f1_metric})

# Path ke gambar yang ingin diuji
image_path = "/content/drive/MyDrive/skripsi/validation/no pedestrian/val (111).jpg"  # Ganti dengan path gambar Anda

# Membaca gambar menggunakan OpenCV
image = cv2.imread(image_path)
img_height, img_width, _ = image.shape

# Melakukan resizing gambar sesuai dengan kebutuhan model
IMG_SIZE = 224
image_resized = cv2.resize(image, (IMG_SIZE, IMG_SIZE))
image_resized = np.expand_dims(image_resized, axis=0)  # Menambahkan dimensi batch
image_resized = image_resized.astype('float32') / 255.0  # Normalisasi

# Melakukan prediksi dengan model
prediction = model.predict(image_resized)

# Menampilkan hasil prediksi
if prediction[0][0] > prediction[0][1]:
    print("Gambar tersebut adalah bukan pejalan kaki.")
else:
    print("Gambar tersebut adalah pejalan kaki.")
